In [1]:
import re

import pandas as pd

from SPARQLWrapper import SPARQLWrapper, JSON
from sqlalchemy import create_engine, MetaData, text
#from sqlalchemy.orm import sessionmaker

In [2]:
#create sqlite engine in memory
engine = create_engine("sqlite:///:memory:")

with engine.connect() as con:
    with open("create_schema.sql") as file:
        #only one query is accepted at a time, so we split - feels hacky
        query_list = file.read().split(';')
        for query_text in query_list:
            query = text(query_text)
            con.execute(query)
con.close()

In [3]:
#check if tables and columns are all there
#m = MetaData()
#m.reflect(engine)
#for table in m.tables.values():
#    print(table.name)
#    for column in table.c:
#        print(column.name)

In [4]:
#wikidata endpoint
sparql = SPARQLWrapper(
    "https://query.wikidata.org/sparql"
)

#query for the art_object_list with AUTO_LANGUAGE replaced
sparql.setQuery("""
    SELECT ?artObject ?artObjectLabel ?artObjectDescription ?ownedby ?ownedbyLabel
    WHERE {
        ?artObject wdt:P31 wd:Q3305213 ; # Get items that are instances of painting
         wdt:P127 ?ownedby. #at least one owner is known in wikidata
        ?ownedby wdt:P1840 wd:Q30335959 . #the owner was ALIU Red Flag   
    
        SERVICE wikibase:label { bd:serviceParam wikibase:language "de, en"}
    }
    GROUP BY ?artObject ?artObjectLabel ?artObjectDescription ?ownedby ?ownedbyLabel
    """
)

sparql.setReturnFormat(JSON) #request json

#query and reshape to create dataframe with field values only
results = sparql.query().convert() 
results_df = pd.json_normalize(results['results']['bindings']) 
results_df = results_df[[col for col in results_df if 'value'in col]]

#change to db column names
results_df.columns = "ID TITLE DESCRIPTION PAINTER_ID PAINTER_NAME".split()
#only use Q### ids
results_df['ID'] = results_df['ID'].apply(lambda uri: uri.strip("http://www.wikidata.org/entity/"))
results_df['PAINTER_ID'] = results_df['PAINTER_ID'].apply(lambda uri: uri.strip("http://www.wikidata.org/entity/"))

In [5]:
results_df

,ID,TITLE,DESCRIPTION,PAINTER_ID,PAINTER_NAME
0,Q1114220,Mohnblumenfeld bei Vétheuil,"Gemälde von Claude Monet, 1880",Q96297,Emil Georg Bührle
1,Q1810348,Le quai malaquais,Gemälde von Camille Pissarro,Q89582,Bruno Lohse
2,Q3793374,Der Blaue Reiter,Gemälde von Wassily Kandinsky,Q96297,Emil Georg Bührle
3,Q4211284,landschap met stenenbrug,Gemälde von Rembrandt,Q329815,Abraham Bredius
4,Q11710021,Homer,Gemälde von Rembrandt,Q329815,Abraham Bredius
...,...,...,...,...,...
625,Q107549104,"A moonlit river landscape with a village, a wi...",Gemälde von Aart van der Neer,Q18639521,Alois Miedl
626,Q108515220,Blick aufs Meer,Gemälde von Pierre-Auguste Renoir im Kunsthaus...,Q19429021,Theodor Fischer
627,Q112331692,Mandolin and Pipe,painting by Juan Gris,Q17277888,Gottlieb Friedrich Reber
628,Q114148816,Auferstehung Christi,Gemälde von Lucas Cranach dem Älteren,Q18704516,Hans Lange


In [6]:
#insert complete table into df
results_df.to_sql('t_art_objects', con=engine, index=False, if_exists='replace')

630

In [7]:
#m.tables['t_art_objects']

In [8]:
#check if data is actually in db
with engine.connect() as con:
    art_ids = con.execute(text("SELECT ID FROM t_art_objects")).fetchall()
con.close()

art_ids = [a[0] for a in art_ids]

In [22]:
#original query from assignment
img_query = """
    SELECT (?onwer as ?onwerID) ?onwerLabel ?ownFrom ?ownUntil
    WHERE {
      wd:Q1114220 wdt:P127 ?onwer.
      OPTIONAL {
        wd:Q1114220 p:P127 [ ps:P127 ?onwer ; pq:P580 ?ownFrom].
      }
      OPTIONAL {
        wd:Q1114220 p:P127 [ ps:P127 ?onwer ; pq:P582 ?ownUntil].
      }
      SERVICE wikibase:label { bd:serviceParam wikibase:language "de","en" }
    }
    """
#mapping 
owner_cols = {'ownUntil.value':'END_OWNERSHIP', 'ownFrom.value':'START_OWNERSHIP', 
              'onwerLabel.value':'OWNER_NAME', 'onwerID.value':'OWNER_ID'}

for ID in art_ids[:15]:
    #query for the art_object_list with AUTO_LANGUAGE replaced
    print(ID)
    local_query = re.sub("Q1114220", ID, img_query)
    #print(local_query)
    sparql.setQuery(local_query)

    results = sparql.query().convert() 
    local_df = pd.json_normalize(results['results']['bindings']) 
    local_df = local_df[[col for col in local_df if 'value'in col]]
    #print(owner_cols[:len(local_df.columns)], local_df.columns)
    #print(local_df.columns)
    local_df.columns = [owner_cols[col] for col in local_df.columns]
    local_df['ID'] = [ID]*len(local_df)
    print(len(local_df))
    
    for col in owner_cols.values():
        if col not in local_df:
            local_df[col] = ""
    if 'OWNER_ID' in local_df:
        local_df['OWNER_ID'] = local_df['OWNER_ID'].apply(lambda uri: uri.strip("http://www.wikidata.org/entity/") if type(uri)==str else uri)
    with engine.connect() as con:
        local_df.to_sql('t_art_owners', con=engine, index=False, if_exists='append')
    con.close()
#print(results)

Q1114220
5
Q1810348
10
Q3793374
2
Q4211284
4
Q11710021
2
Q16579374
3
Q16581524
1
Q16621952
1
Q16937224
1
Q17275978
1
Q17275980
2
Q17275984
1
Q17276052
5
Q17322890
1
Q17323363
1


In [25]:
with engine.connect() as con:
    sample = con.execute(text("SELECT * FROM t_art_owners")).fetchall()
con.close()

In [26]:
sample

[('1883-01-01T00:00:00Z', 'Abraham Bredius', 'Q329815', 'Q17323363', ''),
 ('1956-01-01T00:00:00Z', 'Emil Georg Bührle', 'Q96297', 'Q1114220', '1941-01-01T00:00:00Z'),
 ('1940-01-01T00:00:00Z', 'Max Emden', 'Q1912453', 'Q1114220', '1929-01-01T00:00:00Z'),
 ('1940-01-01T00:00:00Z', 'Hans Heinrich Emden', 'Q90482126', 'Q1114220', '1940-01-01T00:00:00Z'),
 (None, 'Fritz Nathan', 'Q1418856', 'Q1114220', '1940-01-01T00:00:00Z'),
 (None, 'Walter Feilchenfeldt', 'Q2544162', 'Q1114220', '1940-01-01T00:00:00Z'),
 ('1934-01-01T00:00:00Z', 'Samuel Fischer', 'Q95541', 'Q1810348', '1907-03-30T00:00:00Z'),
 ('1938-03-01T00:00:00Z', 'Gottfried Bermann Fischer', 'Q97753', 'Q1810348', '1934-01-01T00:00:00Z'),
 ('1904-01-01T00:00:00Z', 'Lucien Pissarro', 'Q964608', 'Q1810348', '1903-01-01T00:00:00Z'),
 ('1940-05-21T00:00:00Z', 'Nationalsozialistische Deutsche Arbeiterpartei', 'Q7320', 'Q1810348', '1938-03-01T00:00:00Z'),
 ('1907-03-30T00:00:00Z', 'Paul Cassirer', 'Q65351', 'Q1810348', '1904-01-01T00:00:

In [16]:
local_df

,OWNER_NAME,OWNER_ID,ID,END_OWNERSHIP,START_OWNERSHIP
0,Emil Georg Bührle,Q96297,Q16937224,,
